In [117]:
import pandas as pd
import numpy as np
import random
import math 

In [168]:
adj_matrix = np.load("init_flight_capacity.npy")

with open("terminal_name.txt") as f:
    airport_id = f.readlines()
airport_id = [i.replace("\n", "") for i in airport_id]
airport_id
# with open('cityid2airportid.pickle', 'rb') as handle:
#     cityid2airportid = pickle.load(handle)

# with open('cityid2population.pickle', 'rb') as handle:
#     cityid2population = pickle.load(handle)
    
# flights_data = pd.read_csv("flights.csv")
# flights_data

['AAL',
 'AAR',
 'ABI',
 'ABQ',
 'ABV',
 'ABX',
 'ABZ',
 'ACC',
 'ACT',
 'ADE',
 'ADL',
 'AEP',
 'AEX',
 'AGU',
 'AKJ',
 'AKL',
 'ALC',
 'ALP',
 'ALY',
 'AMA',
 'AMM',
 'AMS',
 'ANF',
 'AOJ',
 'AQP',
 'ARI',
 'ARM',
 'ARN',
 'ASJ',
 'ASP',
 'ATH',
 'ATL',
 'AUA',
 'AUH',
 'AUS',
 'AXT',
 'AYT',
 'AZO',
 'BCN',
 'BDB',
 'BDL',
 'BEY',
 'BGI',
 'BHM',
 'BIO',
 'BJZ',
 'BKI',
 'BKK',
 'BLQ',
 'BLR',
 'BME',
 'BMI',
 'BNA',
 'BNE',
 'BOD',
 'BOG',
 'BOS',
 'BRU',
 'BTR',
 'BUD',
 'BUF',
 'BUR',
 'BWI',
 'BZE',
 'CAI',
 'CAN',
 'CBR',
 'CCP',
 'CCS',
 'CDG',
 'CGK',
 'CHA',
 'CHC',
 'CHS',
 'CID',
 'CIX',
 'CJA',
 'CJC',
 'CKG',
 'CLE',
 'CLL',
 'CLO',
 'CLT',
 'CMB',
 'CMH',
 'CMI',
 'CMN',
 'CNF',
 'CNS',
 'COS',
 'CPH',
 'CPT',
 'CQM',
 'CRD',
 'CRP',
 'CTA',
 'CTS',
 'CTU',
 'CUR',
 'CUU',
 'CVG',
 'CZM',
 'DAC',
 'DAL',
 'DAM',
 'DAY',
 'DBV',
 'DCA',
 'DEL',
 'DEN',
 'DKR',
 'DLC',
 'DOH',
 'DPS',
 'DRW',
 'DSM',
 'DTW',
 'DUB',
 'DUR',
 'DXB',
 'EAS',
 'EBB',
 'EDI',
 'EGE',
 'ELP',


In [174]:
cityid2airportid = {2451:["AAR", "AAL"],719:["ABI"] }       
cityid2population = {2451: 237551.0, 719: 693494.0}
cityid_outprob = {2451: {"AAR_PER": 0.4, "AAR_ABI": 0.3, "AAL_ABI":0.3}, 719: {"ABI_AAR": 1.0}}

beta_city = [0.779 for i in range(len(cityid2airportid))]
gamma_city = [0.03 for i in range(len(cityid2airportid))]
beta_flight = np.load("beta.npy") # infection rate
gamma_flight = np.copy(beta_flight)
gamma_flight[gamma_flight != 0] = 0.03 # recovery rate

In [121]:
def ppl_same_city_in_plane(airports, probabilities, infection_num_out):
    # Use the random.choices function to fetch 15 elements
    fetch = random.choices(airports, weights=probabilities, k=infection_num_out)

    # Count the number of times each element appears in the fetch
    infection_num_out_list=[]
    for item in airports:
        infection_num_out_list.append(fetch.count(item))
        
    return airports, infection_num_out_list

elements = ['airport1', 'airport2']
probabilities = [0.66, 0.33]
ppl_same_city_in_plane(elements, probabilities, 10)
for k, v in cityid_outprob.items():
    print(k, ppl_same_city_in_plane(list(v.keys()), list(v.values()), 10))


2451 (['AAR_PER', 'AAR_OSL', 'AAL_OSL'], [4, 4, 2])
719 (['OSL_AAR'], [10])


In [176]:
def ppl_same_city_in_plane(airports, probabilities, infection_num_out):
    # Use the random.choices function to fetch 15 elements
    fetch = random.choices(airports, weights=probabilities, k=int(infection_num_out))

    # Count the number of times each element appears in the fetch
    infection_num_out_list=[]
    for item in airports:
        infection_num_out_list.append(fetch.count(item))
        
    return infection_num_out_list

def update_infection(arrive_infection_matrix):
    update_dictionary = dict(zip(airport_id, np.sum(arrive_infection_matrix, axis=0)))
    return update_dictionary

def get_infection_input(infection_dictionary):   
    city_infection_input = dict.fromkeys(cityid2airportid.keys(), 0)
    for k, v in cityid2airportid.items():
        infection_num = []
        for i in v:
            infection_num.append(infection_dictionary[i])
        city_infection_input[k] = sum(infection_num)
    return city_infection_input

def city_infection(city_infection_input, city_recovery):
    city_infection_num = np.array(list(city_infection_input.values()))
    city_population = np.array(list(cityid2population.values()))
    y_c = [(city_population - city_infection_num)/city_population, city_infection_num/city_population, city_recovery]
    
    s, i ,r = SIR(y_c, beta_city, gamma_city)
    
    after_infection = np.ceil(i*city_population)

    return r, after_infection


def flight_infection(flight_adj, flight_infection_num):

    y_f = [np.nan_to_num((flight_adj-flight_infection_num)/flight_adj,nan = 1),\
           np.nan_to_num(flight_infection_num/flight_adj, nan = 0),\
           np.zeros(len(flight_adj))]
    
    
    s, i ,r = SIR(y_f, beta_flight, gamma_flight)
    arrive_infection = np.ceil(i*flight_adj)
    
    infection_ppl_flight = update_infection(arrive_infection)
    
    return infection_ppl_flight

def get_on_plane(flight_infection_matrix, infection_ppl_in_airport):
    for (k, v), num_of_people in zip(cityid_outprob.items(), infection_ppl_in_airport):
        people_on_plane = ppl_same_city_in_plane(list(v.keys()), list(v.values()), num_of_people)
        flight_infection_matrix = update_flight_infection(flight_infection_matrix, list(v.keys()), people_on_plane)
    return flight_infection_matrix
    
def fly_infection_ppl(city_infection_ppl):
    return np.ceil(city_infection_ppl/list(cityid2population.values()))
    

def SIR(y, beta, gamma):
    S, I, R = y 
    d0 = S - beta*S*I # derivative of S(t)
    d1 = I + beta*S*I - gamma*I # derivative of I(t)
    d2 = R+ gamma*I # derivative of R(t)
    
    return [d0, d1, d2]

def update_flight_infection(flight_infection, flight_com, ppl_num_list):
    flight_com = [i.split("_") for i in flight_com]
    for (flight_from, flight_to), infection_num in zip(flight_com, ppl_num_list):
        flight_from_id = airport_id.index(flight_from)
        flight_to_id = airport_id.index(flight_to)
        flight_infection[flight_from_id][flight_to_id] += infection_num
        
    return flight_infection
    

def loop_infection(n_iter, flight_adj):
    
    flight_infection_num = np.zeros((len(flight_adj), len(flight_adj)))
    flight_infection_num[0][2] = 1
    
    city_recovery = [0.03 for i in range(len(cityid2airportid))]
    infection_ppl_list = [1]
    for _ in range(n_iter):
        infection_ppl_flight = flight_infection(flight_adj, flight_infection_num)
        infection_ppl_city = get_infection_input(infection_ppl_flight)
        city_recovery, infection_ppl = city_infection(infection_ppl_city, city_recovery)
        flight_infection_num = get_on_plane(np.zeros((len(flight_adj), len(flight_adj))), fly_infection_ppl(infection_ppl))

        infection_ppl_list.append(np.sum(infection_ppl))
    return flight_infection_num, infection_ppl_list
        


adj_matrix = np.zeros((427,427))
adj_matrix[0][2]=9
adj_matrix
beta_flight = np.zeros((427,427))
beta_flight[0][2]=0.18

gamma_flight = np.copy(beta_flight)
gamma_flight[gamma_flight != 0] = 0.03 # recovery rate

loop_infection(1, adj_matrix)


/opt/anaconda3/envs/ns/lib/python3.7/site-packages/ipykernel_launcher.py:39: RuntimeWarning: invalid value encountered in true_divide
/opt/anaconda3/envs/ns/lib/python3.7/site-packages/ipykernel_launcher.py:40: RuntimeWarning: invalid value encountered in true_divide


(array([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 1., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]]),
 [1, 4.0])

In [44]:
SIR([np.array([0.99, 0.95]), np.array([0.1, 0.04]), np.array([0, 0.005])], np.array([0.779 for i in range(2)]), np.array([0.03 for i in range(2)]))

[array([0.912879, 0.920398]),
 array([0.174121, 0.068402]),
 array([0.003 , 0.0062])]

In [38]:
np.array([0.99, 0.95])*np.array([0.779, 0.779])

array([0.77121, 0.74005])